# README
How to install Jupyter: https://jupyter.org/install \
Where to download the data: https://github.com/patiegm/Udacity_Data_Analysis_Nanodegree/tree/master/Analyze%20AB%20Test%20Results
# Context
Let's assume we run an experiment by randomly assigning users to treatment vs. control. While the users in treatment were exposed to the new page, the users in control still saw the old page. We want to understand whether the new page significantly improved user conversion defined by the percentage of users who converted after being exposed to the new vs. old page.

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import t
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.power as smp

# Load Data

In [2]:
data = pd.read_csv('ab_data.csv')

In [3]:
data

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


# Clean Data

In [4]:
def check_mixed_assignment(df):
    df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index() 
    # count the unique number of groups that a user was assigned to
    df2 = df1[df1['group'] > 1]['user_id'].count() 
    # count the number of users assigned to both groups
    print(df2)

In [5]:
check_mixed_assignment(data)

1895


In [6]:
def remove_mixed_assignment(df):
    df1 = df[['user_id', 'group']].groupby(['user_id']).nunique().reset_index() 
    # count the unique number of groups that a user was assigned to
    df2 = pd.merge(df, df1, on=['user_id'], how='left') 
    return df2[df2['group_y'] == 1][['user_id', 'timestamp', 'group_x', 'landing_page', 'converted']]\
        .rename(columns={'group_x':'group'})
    # only return users assigned to either treatment or control

In [7]:
data1 = remove_mixed_assignment(data)

In [8]:
check_mixed_assignment(data1)

0


In [9]:
def check_exposure_bugs(df):
    print(df[(df['group'] == 'control')&(df['landing_page'] == 'new_page')]['user_id'].count()) 
    # count the number of users in control expposed to treatment
    print(df[(df['group'] == 'treatment')&(df['landing_page'] == 'old_page')]['user_id'].count()) 
    # count the number of users in treatment expposed to control

In [10]:
check_exposure_bugs(data1)

1007
991


In [11]:
def remove_exposure_bugs(df):
    df1 = df[(df['group'] == 'control')&(df['landing_page'] == 'new_page')][['user_id', 'group']] 
    # identify the users in control expposed to treatment
    df2 = df[(df['group'] == 'treatment')&(df['landing_page'] == 'old_page')][['user_id', 'group']] 
    # identify the users in treatment expposed to control
    df3 = pd.concat([df1, df2])
    df4 = pd.merge(df, df3, on=['user_id'], how='left')
    return df4[df4['group_y'].isna()][['user_id', 'timestamp', 'group_x', 'landing_page', 'converted']]\
        .rename(columns={'group_x':'group'})
    # only return users with the correct exposure

In [12]:
data2 = remove_exposure_bugs(data1)

In [13]:
check_exposure_bugs(data2)

0
0


In [14]:
def check_multiple_exposures(df):
    df1 = df[['user_id', 'group']].groupby(['user_id']).count().reset_index() 
    # count the number of exposures that a user had
    df2 = df1[df1['group'] > 1]['user_id'].count() 
    # count the number of users that had multiple exposures
    print(df2)

In [15]:
check_multiple_exposures(data2)

1


In [16]:
def consolidate_multiple_exposures(df):
    df1 = df.groupby(['user_id', 'group', 'landing_page'])\
        .agg({'timestamp': ['min', 'max'], 'converted': ['count', 'sum']}) 
    # get the timestamps of the first and last exposure, the number of exposures and the number of conversions
    df1.columns = df1.columns.droplevel(0)
    df2 = df1.reset_index()
    df2['converted'] = df2.apply(lambda x: int(x['sum'] > 0), axis=1) # 1 if the user has one conversion
    df2['conversion_rate'] = 1.0*df2['sum']/df2['count'] # the number of conversions divided by the number of exposures
    return df2
    # one user will only have one row

In [17]:
data3 = consolidate_multiple_exposures(data2)

In [18]:
check_multiple_exposures(data3)

0


# Sanity Check 

In [19]:
def check_any_assignment_imbalance(df):
    df1 = df[['user_id', 'group']].groupby(['group']).count().reset_index() 
    # count the number of users in treatment vs. control
    print(df1)
    pvalue = stats.binom_test(df1[df1['group'] == 'treatment']['user_id'].values[0], 
                              n=df1['user_id'].sum(),
                              p=0.5, 
                              alternative='two-sided') 
    # test whether the treatment vs. control has the equal sample size
    print(pvalue)

In [20]:
check_any_assignment_imbalance(data3)

       group  user_id
0    control   143293
1  treatment   143398
0.8459923345444519


# Hypothesis Testing

In [21]:
def calculate_pvalue(df):
    n_treatment = df[df['group'] == 'treatment']['user_id'].count() # the number of users in treatment
    n_control = df[df['group'] == 'control']['user_id'].count() # the number of users in control
    
    p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/n_treatment 
    # the probability of a user in treatment to convert
    p_control = 1.0*df[df['group'] == 'control']['converted'].sum()/n_control 
    # the probability of a user in control to convert
    
    var_treatment = p_treatment*(1-p_treatment) # the variance of the probability of a user in treatment to convert
    var_control = p_control*(1-p_control) # the variance of the probability of a user in treatment to convert
    
    p_delta = p_treatment - p_control # the delta of the probability of a user to convert in treatment vs. control
    print(p_delta)
    
    pooled_se = np.sqrt(var_treatment/n_treatment + var_control/n_control) # the pooled standard error of the t test
    t_statistic = p_delta/pooled_se # the t statistic
    dof = (var_treatment/n_treatment + var_control/n_control)**2\
    /(var_treatment**2/(n_treatment**2*(n_treatment-1)) + var_control**2/(n_control**2*(n_control-1)))
    # the degree of freedom
    pvalue = 2*t.cdf(-abs(t_statistic), dof) # the p value of the t test
    print(pvalue)
    
    lower = p_delta - t.ppf(0.975, dof)*pooled_se # the lower bound of the confidence interval
    upper = p_delta + t.ppf(0.975, dof)*pooled_se # the upper bound of the confidence interval
    print(lower)
    print(upper)

In [22]:
calculate_pvalue(data3)

-0.0014478458686042056
0.23202039224946788
-0.0038221778568021604
0.0009264861195937492


# Power Calculation

In [23]:
def calculate_mde(df):
    n_treatment = df[df['group'] == 'treatment']['user_id'].count() # the number of users in treatment
    n_control = df[df['group'] == 'control']['user_id'].count() # the number of users in control
    
    power_analysis = smp.TTestIndPower()
    
    effect_size = power_analysis.solve_power(
        nobs1=n_control, power=0.8, alpha=0.05, ratio=1.0*n_treatment/n_control, alternative='two-sided'
    ) 
    p = 1.0*df['converted'].sum()/df['user_id'].count() # the probability of a user to convert
    sd = np.sqrt(p*(1-p)) # the standard deviation of a user to convert
    mde = effect_size*sd # the minimum detectable effect with the current sample size
    print(mde)
    
    p_treatment = 1.0*df[df['group'] == 'treatment']['converted'].sum()/n_treatment 
    # the probability of a user in treatment to convert
    p_control = 1.0*df[df['group'] == 'control']['converted'].sum()/n_control 
    # the probability of a user in control to convert
    p_delta = p_treatment - p_control 
    # the measured delta of the probability of a user to convert in treatment vs. control
    nobs1 = power_analysis.solve_power(
        effect_size=1.0*p_delta/sd, power=0.8, alpha=0.05, ratio=1.0, alternative='two-sided'
    )
    # the required sample size when setting the currently measured delta as the minimum detectable effect
    print(nobs1)

In [24]:
calculate_mde(data3)

0.003393909728661452
787644.1019180008


# Validation

In [25]:
X = pd.get_dummies(data3['group'], drop_first=True)
y = data3['converted']

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              converted   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.428
Date:                Fri, 11 Nov 2022   Prob (F-statistic):              0.232
Time:                        15:29:33   Log-Likelihood:                -83972.
No. Observations:              286691   AIC:                         1.679e+05
Df Residuals:                  286689   BIC:                         1.680e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1202      0.001    140.266      0.0